In [1]:
import re
import nltk
import string
import pandas as pd
import numpy as np
from pprint import pprint


# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, TfidfModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
sw = open("../src/stopwords.txt", "r")
stopwords = sw.read()
stopwords_list = stopwords.split(", ")
stopwords_set = set(stopwords_list)
# sw.close()

In [3]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(stopwords_list)

In [4]:
regular_episodes = pd.read_csv("../data/jeopardy_regular_episodes.csv")
regular_episodes = regular_episodes.sample(frac=.05)
type(regular_episodes['Question and Answer'])

pandas.core.series.Series

In [5]:
# Convert to list
clues = regular_episodes['Question and Answer'].values.tolist()

# Remove Emails
clues = [re.sub('\S*@\S*\s?', '', sent) for sent in clues]

# Remove new line characters
clues = [re.sub('\s+', ' ', sent) for sent in clues]

# Remove distracting single quotes
clues = [re.sub("\'", "", sent) for sent in clues]

In [6]:
#Generator function for tokenizing and removing punctuation
# deacc=True removes punctuation

def tokenize(clues):
    for clue in clues:
        yield(gensim.utils.simple_preprocess(str(clue), deacc=True))

In [7]:
clue_words = list(tokenize(clues))

In [9]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(clue_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[clue_words], threshold=100)

In [10]:
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[clue_words[0]]])

['skivvies', 'underwear', 'at', 'one', 'time', 'trademark']


In [11]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts, stop_words):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    nlp = spacy.load('en', disable=['parser', 'ner'])
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [14]:
# Remove Stop Words and returns a list of tokens as long as clue_words are tokenized 
nostops = remove_stopwords(clue_words, stop_words)

# Form Bigrams
clue_words_bigrams = make_bigrams(nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
# clue_lemmatized = lemmatization(clue_words_bigrams,allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# Lemmatize without the allowed_postages
clue_lemmatized = lemmatization(clue_words_bigrams)
clue_lemmatized = remove_stopwords(clue_lemmatized, stop_words)



In [16]:
print(clue_lemmatized[:10])

[['skivvy', 'underwear', 'time', 'trademark'], ['well', 'love', 'music', 'try'], ['air', 'invisible', 'emanation'], ['chinese', 'player', 'pick'], ['fan', 'look', 'rafter', 'see', 'retire', 'jersey'], ['auction', 'card', 'game', 'twice', 'sell'], ['cross', 'mogadishu', 'maldive'], ['accident', 'run', 'street'], ['star', 'cbstv', 'launch', 'shortlived', 'product'], []]


In [17]:
# Create Dictionary
id2word = corpora.Dictionary(clue_lemmatized)

# Create Corpus
texts = clue_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
#tfidf
# for doc in corpus:
#     print([[id2word[idx], freq] for idx, freq in doc])



In [18]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('skivvy', 1), ('time', 1), ('trademark', 1), ('underwear', 1)]]

BUILD THE LDA MODEL

In [19]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=13, 
                                           random_state=123,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [21]:
# Print the keywords in the 10 topics

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

# the weights of top 10 keywords that contribute to each
# topic reflect how important that keyword is to that topic

[(0,
  '0.048*"line" + 0.042*"night" + 0.037*"tv" + 0.033*"team" + 0.025*"put" + '
  '0.025*"baby" + 0.024*"ad" + 0.023*"kid" + 0.020*"always" + 0.019*"brand"'),
 (1,
  '0.080*"write" + 0.047*"film" + 0.033*"high" + 0.026*"later" + 0.025*"great" '
  '+ 0.023*"family" + 0.020*"early" + 0.020*"bird" + 0.019*"live" + '
  '0.018*"inspire"'),
 (2,
  '0.096*"begin" + 0.033*"singer" + 0.031*"dog" + 0.028*"character" + '
  '0.025*"war" + 0.022*"eye" + 0.021*"dry" + 0.018*"doctor" + 0.018*"talk" + '
  '0.017*"draw"'),
 (3,
  '0.119*"play" + 0.071*"find" + 0.037*"found" + 0.030*"popular" + '
  '0.022*"mind" + 0.022*"start" + 0.021*"kind" + 0.019*"learn" + 0.018*"dress" '
  '+ 0.017*"version"'),
 (4,
  '0.022*"die" + 0.020*"help" + 0.019*"music" + 0.018*"song" + 0.016*"love" + '
  '0.016*"end" + 0.016*"base" + 0.016*"form" + 0.015*"number" + 0.015*"send"'),
 (5,
  '0.048*"come" + 0.029*"bear" + 0.028*"old" + 0.025*"part" + 0.018*"top" + '
  '0.018*"cover" + 0.017*"foot" + 0.016*"face" + 0.015*"fl

In [22]:
#Perplexity = a measure of how good the model is. 
# The lower, the better
print ("Perplexity:", lda_model.log_perplexity(corpus))

Perplexity: -13.681734704153873


In [24]:
coherence_model = CoherenceModel(model=lda_model, texts=clue_lemmatized,
                     dictionary=id2word, coherence='c_v')

coherence_lda = coherence_model.get_coherence()

print("Coherence Score:", coherence_lda)

Coherence Score: 0.6051870204481309


In [25]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.322883 -0.230588       1        1  16.509513
5     -0.205407  0.335239       2        1  15.401275
12     0.012713 -0.003390       3        1   9.926285
11     0.029233 -0.006717       4        1   9.335778
1      0.025824 -0.006011       5        1   9.163653
7      0.033243 -0.007461       6        1   9.110356
3      0.059565 -0.011480       7        1   5.590200
6      0.061178 -0.011666       8        1   5.065816
8      0.058536 -0.011372       9        1   4.966015
10     0.060742 -0.011626      10        1   4.639735
2      0.062065 -0.011736      11        1   4.462909
0      0.062394 -0.011766      12        1   4.077597
9      0.062797 -0.011427      13        1   1.750867, topic_info=                Term        Freq       Total Category  logprob  loglift
125             play  376.000000  376.000000  Default  30.0000  30.0000
72             write  415.000000  415.000000  Default  29.0000  29.0000
253             come  419.000000  419.000000  Default  28.0000  28.0000
189            begin  243.000000  243.000000  Default  27.0000  27.0000
1               time  303.000000  303.000000  Default  26.0000  26.0000
...              ...         ...         ...      ...      ...      ...
11567       asbestos    0.052859    1.205563  Topic13  -9.8371   0.9180
4356   heatresistant    0.052859    1.205565  Topic13  -9.8371   0.9180
4173         project    0.053130    1.220992  Topic13  -9.8320   0.9104
1418         mineral    0.052866    1.206101  Topic13  -9.8370   0.9177
3969           meter    0.053134    1.258142  Topic13  -9.8319   0.8805

[423 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
982       8  0.953120  abbreviate
1493     10  0.984595      accept
377      11  0.957214      accord
318       5  0.989072       actor
932       7  0.952239     actress
...     ...       ...         ...
731       3  0.994585       world
661       2  0.986772       would
72        5  0.996564       write
3154     11  0.941843      yellow
805       6  0.986843       young

[379 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 6, 13, 12, 2, 8, 4, 7, 9, 11, 3, 1, 10])

In [ ]:
mallet_path = "~/Downloads/mallet-2.0.8/bin/mallet"
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path,
                                             corpus=corpus, 
                                             num_topics=13,
                                             id2word=id2word)

In [ ]:
# Show the topics
pprint(ldamallet.show_topics(formatted=False)

In [ ]:
coherence_model_ldamallet = CoherenceModel(model=ldamallet,
                                          text=clue_lemmatized,
                                          dictionary=id2word,
                                          coherence='c_v')

coherence_ldamallet = coherence_model_ldamallet.get_coherence()

print("Coherence Score:", coherence_ldamallet)

## picking the best number of topics for the LDA model

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=clue_lemmatized, start=2, limit=40, step=6)